In [ ]:
from datetime import datetime
from heapq import nlargest
from operator import itemgetter
from collections import defaultdict
from sys import stdout
print('Preparing arrays...')

In [ ]:
###### finding most popular 50
f = open("train.csv", "r")
f.readline()
most_hotels_cntr_hm_sdi = defaultdict(int)
total = 0

In [ ]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '':
        most_hotels_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)] += 1

f.close()
end = time.time()
print(end - start)

In [ ]:
topitems = nlargest(50, most_hotels_cntr_hm_sdi.items(), key=itemgetter(1))

In [ ]:
topitems_dict = defaultdict(int)

In [ ]:
for k,v in topitems:
    topitems_dict[k] = v

In [ ]:
############ Features transformation & Construction #############
f = open("train.csv", "r")
f.readline()
feature1_cntr_hm_sdi = defaultdict(lambda: defaultdict(list)) # feature_contstrunction
total = 0
missline = 0

In [ ]:
import time

start = time.time()

while 1:
    line = f.readline().strip()
    total += 1

    if total % 10000000 == 0:
        print('Read {} lines...'.format(total))

    if line == '':
        break

    arr = line.split(",")
    srch_destination_id = arr[16]
    hotel_country = arr[21]
    hotel_market = arr[22]

    if srch_destination_id != '' and hotel_country != '' and hotel_market != '' and ((hotel_country,hotel_market,srch_destination_id) in topitems_dict):
        try:
            search_time_DT = datetime.strptime(arr[0], '%Y-%m-%d %H:%M:%S')
            arr[0] = search_time_DT.hour      #srch_time_hour
            arr[1] = search_time_DT.weekday() #srch_time_weekday
            #arr[2] = int(arr[2])             #posa_continent
            
            arr[6] = float(arr[6]) #orig_dest_distance
        
            #arr[8] = int(arr[8]) #is_mobile
            #arr[9] = int(arr[9]) #is_package
            #arr[10] = int(arr[10]) #channel
        
            checkInDateDT = datetime.strptime(arr[11], "%Y-%m-%d")
            checkOutDateDT = datetime.strptime(arr[12], "%Y-%m-%d") 
            arr[11] = checkInDateDT.timetuple().tm_yday + 1  #srch_ci_day
            arr[12] = checkOutDateDT.timetuple().tm_yday+ 1 #srch_co_day

            arr[13] = int(arr[13]) #srch_adult_count
            arr[14] = int(arr[14]) #srch_child_count
            arr[15] = int(arr[15]) #srch_room_count
            
            arr[16] = arr[12] - arr[11] #duration
            #arr[17] = int(arr[17]) #srch_dest_type_id
            arr[18] = int(arr[18])  #is_booking should be weighted
            arr[19] = int(arr[19])  #hotel_cluster
        except:
            missline += 1
            continue
        colNames = ['hour','dow','posa','','','','dist','','mobile','package','channel','ci','co',
                    'adults','children','room','stay','type','booking','cluster']
        colindex = [0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19]
        for i in colindex:
            feature1_cntr_hm_sdi[(hotel_country,hotel_market,srch_destination_id)][colNames[i]].append(arr[i])

f.close()
end = time.time()
print(end - start)